In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# (UPDATED to match your file/class)
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
# (Uses a safe fallback if the file isn't present so the app never crashes)
logo_path = "Grazioso_Salvare_Logo.png"  # put the provided logo file in the working directory
encoded_image = None
if os.path.exists(logo_path):
    with open(logo_path, "rb") as f:
        encoded_image = base64.b64encode(f.read()).decode()

# Simple helper to render the logo (if available)
logo_img = html.Img(
    src=f"data:image/png;base64,{encoded_image}" if encoded_image else "",
    style={"height": "60px"} if encoded_image else {},
    **({"title": "Grazioso Salvare"} if encoded_image else {})
)

# Unique identifier with your name
uid = html.Div("Dashboard Author: Dusty Cook", style={"fontStyle": "italic", "fontSize": "0.9rem"})

# Rescuer filter options required by the spec
filter_options = [
    {"label": "Water Rescue", "value": "water"},
    {"label": "Mountain or Wilderness Rescue", "value": "mountain"},
    {"label": "Disaster or Individual Tracking", "value": "disaster"},
    {"label": "Reset (All)", "value": "reset"},
]

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Div([
            logo_img,
            uid
        ], style={"display": "flex", "gap": "1rem", "alignItems": "center", "justifyContent": "center"})
    ]),
    html.Hr(),
    html.Div(
        children=[
            # FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
            html.Div([
                html.Label("Rescue Type Filter"),
                dcc.RadioItems(
                    id="filter-type",
                    options=filter_options,
                    value="reset",                 # default = unfiltered view
                    inline=True
                )
            ], style={"textAlign": "center"})
        ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        # If you completed the Module Six Assignment, you can copy in the code you created here 
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        style_table={"height": "450px", "overflowY": "auto"},
        style_cell={"minWidth": 120, "maxWidth": 220, "whiteSpace": "normal"},
        style_header={"fontWeight": "bold"}
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'gap': '1rem'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={"flex": 1}
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={"flex": 1}
        )
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Helper: build Mongo query from rescue type (matches SNHU spec table)
def build_query(filter_type: str) -> dict:
    """
    Returns a MongoDB query for the requested rescue type.
    These follow the commonly provided Dashboard Specifications for CS-340.
    """
    if filter_type == "water":
        # Water Rescue
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        return {
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == "mountain":
        # Mountain or Wilderness Rescue
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        return {
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == "disaster":
        # Disaster or Individual Tracking
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        return {
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    # Reset / default: no filter
    return {}

# DataTable data update based on selected rescue filter
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    query = build_query(filter_type or "reset")
    records = db.read(query)
    # keep columns consistent and remove _id if present
    if records and isinstance(records, list):
        for r in records:
            r.pop("_id", None)
    # If no records (or on error), fall back to all to avoid an empty initial state
    if not records:
        records = db.read({})
        for r in records:
            r.pop("_id", None)
    return records

# Display a chart based on what's currently in the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame(viewData) if viewData is not None else df.copy()
    if dff.empty:
        return [html.Div("No data to chart.")]
    # Simple bar chart: Top breeds by count
    top = (
        dff.groupby("breed", dropna=False)
           .size()
           .reset_index(name="count")
           .sort_values("count", ascending=False)
           .head(10)
    )
    fig = px.bar(top, x="breed", y="count", title="Top Breeds in Current Result (Top 10)")
    fig.update_layout(margin=dict(l=20, r=20, t=60, b=20), xaxis_title=None)
    return [dcc.Graph(figure=fig)]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return [html.Div("No map data.")]
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if not index:
        row = 0
    else:
        row = index[0]

    # Guard against out-of-range or missing lat/lon columns
    try:
        lat = dff.iloc[row, 13]
        lon = dff.iloc[row, 14]
        breed = dff.iloc[row, 4] if 4 < dff.shape[1] else "Unknown"
        name  = dff.iloc[row, 9] if 9 < dff.shape[1] else "Unknown"
    except Exception:
        return [html.Div("Selected row missing coordinates in the dataset.")]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H4("Animal Name"),
                    html.P(str(name))
                ])
            ])
        ])
    ]

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app,
# the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server()

Dash app running on https://lakeuranium-culturesaga-3000.codio.io/proxy/8050/
